In [1]:
from stackapi import StackAPI
import pandas as pd
import numpy as np
import time
import datetime
import json
from firebase import firebase

In [2]:
def get_id(x):
    if type(x) == dict:
        return x['user_id']
    else:
        return x

In [3]:
config_file = '../data/APIData/api_params'

In [4]:
with open(config_file) as conf:
    configs = json.load(conf)

In [17]:
last_request = configs['last_request']
SITE = StackAPI('superuser', key="nIFln5DrNi7grh*o22xAIw((")
SITE.page_size = configs['page_size']
SITE.max_pages = configs['max_pages']
api_filter = configs['filter']

# Get Posts and Tags

In [7]:
request_time = int(time.time())
last_request, request_time

(1614741944, 1614997616)

In [8]:
# Instead of max_size and max_pages, would use fromdate= the last request in unix time
posts = SITE.fetch('posts', filter=api_filter, fromdate=last_request, todate= request_time)

In [10]:
posts

{'backoff': 0,
 'has_more': False,
 'page': 7,
 'quota_max': 10000,
 'quota_remaining': 10000,
 'total': 0,
 'items': [{'owner': {'user_id': 1281710},
   'score': 0,
   'creation_date': 1614995841,
   'post_type': 'question',
   'post_id': 1631232,
   'title': 'Swapping machine OS with VM OS',
   'body_markdown': 'For several months I&#39;ve been playing around on VirtualBox with Linux Mint 20.0 installed on it and have tweaked the OS just the way I now want it.  \r\nThis VB resides inside of main bare metal Windows 7 Pro. \r\n8 gb ram allocated for each OS.\r\n\r\nI would like to swap Linux Mint (on VB) with the main bare metal Win7 OS.\r\nAnd when the Win7 OS is eventually resident on VB then it will be disconnected from the modem, and only used to run old software when needed.\r\n\r\nSo, Linux Mint 20.0 will become my new go-to OS on the main bare metal desktop computer.\r\n\r\nHow can this transfer be most easily accomplished?',
   'link': 'https://superuser.com/q/1631232',
   'bod

In [18]:
full = pd.DataFrame(posts)
items = pd.DataFrame(full['items'].tolist())
items['owner'] = items['owner'].apply(get_id)
items['creation_date'] = pd.to_datetime(items['creation_date'], unit='s')
answered_questions = items[(items['post_type'] == "answer") & (items['score'] > 2)].post_id
items = items[~items['post_id'].isin(answered_questions)]

In [19]:
tags = SITE.fetch('questions', filter='!BHMIb2uwAoY2iaeLXw*o8d5gWzG74D', fromdate=last_request, todate=request_time)

In [20]:
tags_data = pd.DataFrame(pd.DataFrame(tags)['items'].tolist())
tags_data

,tags,question_id
0,"[windows-7, virtualbox, linux-mint, swap, bare...",1631232
1,"[windows, snap]",1630983
2,"[google-chrome, android]",1631233
3,"[windows-10, boot]",1631231
4,"[windows, certificate, certutil]",1631230
...,...,...
369,"[networking, router, port-forwarding]",1630455
370,"[google-chrome, kiosk]",1630460
371,"[boot, motherboard, bios]",1630453
372,"[windows-7, desktop]",1630448


In [22]:
last_request = request_time 
last_request

1614997616

In [23]:
api_data = items.merge(tags_data, how='left', left_on='post_id', right_on='question_id').drop('question_id', axis=1)

In [24]:
api_data

,owner,score,creation_date,post_type,post_id,title,body_markdown,link,body,tags
0,1281710.0,0,2021-03-06 01:57:21,question,1631232,Swapping machine OS with VM OS,For several months I&#39;ve been playing aroun...,https://superuser.com/q/1631232,<p>For several months I've been playing around...,"[windows-7, virtualbox, linux-mint, swap, bare..."
1,1079976.0,0,2021-03-06 02:19:11,answer,1631234,Swapping machine OS with VM OS,You would need to P to V the Windows 7 machine...,https://superuser.com/a/1631234,<p>You would need to P to V the Windows 7 mach...,NaN
2,1281351.0,1,2021-03-05 01:37:42,question,1630983,Windows 10 - Cannot Move Window Partially Off-...,"I am not sure how this might have happened, bu...",https://superuser.com/q/1630983,"<p>I am not sure how this might have happened,...","[windows, snap]"
3,933117.0,0,2021-03-06 01:27:43,answer,1631229,What is a RAM Bank? How is it defined?,The division of a DDR4 memory address on moder...,https://superuser.com/a/1631229,<p>The division of a DDR4 memory address on mo...,NaN
4,1031687.0,0,2021-03-06 02:01:24,question,1631233,Does Google Chrome&#39;s on Android &#39;Lite ...,If I install a PWA onto my Android phone using...,https://superuser.com/q/1631233,<p>If I install a PWA onto my Android phone us...,"[google-chrome, android]"
...,...,...,...,...,...,...,...,...,...,...
679,292106.0,0,2021-03-03 04:32:08,question,1630444,Restore PFILE files copied directly off of a B...,I have PFILE files copied directly off of a Bi...,https://superuser.com/q/1630444,<p>I have PFILE files copied directly off of a...,"[bitlocker, recovery]"
680,510662.0,1,2021-03-03 04:07:04,answer,1630441,Run Powershell with parameters from batch file,Batch script that accepts arguments\r\n-\r\n\r...,https://superuser.com/a/1630441,<h2>Batch script that accepts arguments</h2>\n...,NaN
681,226999.0,1,2021-03-03 04:01:55,answer,1630440,Does Keeping a Cookie &quot;Whitelist&quot; Sl...,If you open developer tools and go to the *Net...,https://superuser.com/a/1630440,<p>If you open developer tools and go to the <...,NaN
682,1280499.0,0,2021-03-03 03:46:36,answer,1630439,Could not connect Ethernet0 to virtual network...,First check if your vmware services are runnin...,https://superuser.com/a/1630439,<p>First check if your vmware services are run...,NaN


# Save Data to File and Update Configs

In [25]:
today = datetime.datetime.now()
unique_second = str(today.year) + str(today.month).zfill(2) + str(today.day) + str(today.hour).zfill(2) + str(today.minute).zfill(2) + str(today.second).zfill(2)
unique_second

'2021035183012'

In [26]:
api_data.to_csv("../data/APIData/" + unique_second + ".csv")

In [27]:
test_file = {
    "page_size": 100,
    "max_pages": 100,
    "last_request": last_request,
    "filter": "!0S2DU7n2**TqdL2snKOmnaNHL"
}

In [28]:
with open(config_file, 'w') as conv: 
     conv.write(json.dumps(test_file))